In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as ipw
import matplotlib.pyplot as plt
import vpython as vp

from scipy.linalg import expm
from random import choice

output_notebook()

In [ ]:
# Define a function to map a 1D array to a 2D/3D lattice, only works on lattice of equal-lateral size in Cartesian system
def map(index, size, dimension): 
    if dimension == 1:
        return [index, 0, 0];
    if dimension == 2:
        x = index%size;
        y = np.floor(index/size);
       # return vp.vector(x,y,0);
        return [x, y, 0];
    if dimension == 3:
        sub_index = index%(size**2);
        x = sub_index%size;
        y = np.floor(sub_index/size);
        z = np.floor(index/size**2);
       # return vp.vector(x,y,z)
        return [x, y, z];

In [ ]:
# Define a function to map the 3D vector back into the 1D array index
def antimap(array, size, dimension):
    if dimension == 1:
        return array[0];
    if dimension == 2:
        return array[0] + array[1]*size;
    if dimension == 3:
        return array[0] + array[1]*size + array[2]*size*size;

In [ ]:
# Define a function to find the index of the nearest neighbours
def nNbr(index, size, dimension):
    if dimension == 1:
        return [(index+size-1)%size, (index+size+1)%size]; # apply periodic condition
    
    if dimension == 2:
        pos = map(index,size,dimension);
        north = np.mod(np.add(pos,[0,size+1,0]), size);
        south = np.mod(np.add(pos,[0,size-1,0]), size);
        east = np.mod(np.add(pos,[size+1,0,0]), size);
        west = np.mod(np.add(pos,[size-1,0,0]),size);
        return [antimap(north, size, 2), antimap(south, size, 2), antimap(east, size, 2), antimap(west, size, 2)];
    
    if dimension == 3:
        pos = map(index,size,dimension);
        north = np.mod(np.add(pos,[0,size+1,0]), size);
        south = np.mod(np.add(pos,[0,size-1,0]), size);
        east = np.mod(np.add(pos,[size+1,0,0]), size);
        west = np.mod(np.add(pos,[size-1,0,0]),size);
        up = np.mod(np.add(pos,[0,0,size+1]), size);
        down = np.mod(np.add(pos,[0,0,size-1]), size);
        return [antimap(north, size, 3), antimap(south, size, 3), antimap(east, size, 3), antimap(west, size, 3), antimap(up, size, 3), antimap(down, size, 3)];

In [ ]:
# Set up the system parameters
temp = 150; # in [K] 
L = 10; # Define the lattice size
dims = 2; # Define the dimensionality of the lattice
N = L**dims; # Compute the total number of the sites
kB = 8.617E-5 # Boltzmann constant in [eV/K]
J = -4.0*kB; # Define the coupling strength between spins
beta = 1/(temp*kB);
MaxIteration = 1000; # Define the maximum amount of iteration
r = 0.2; # dot radius for visualization

In [ ]:
# Set up array containers for data manipulation and initial conditions
#lattice = np.empty((N*N,3),int); # Create a container for the lattice sites
Apos = [None]*N; # Create a container for site positions using vpython.vector()
#spin = np.onesN; # Create a container for the spins for all the lattice sites
spinV = np.empty(N,int);
spinC = [None]*N;
nIdx = np.empty((N,dims*2), np.intp); # Create a container to store the indices of the nearest neighbours of each site
probs = np.empty(N,np.float);
for ii in range(N):
    #lattice[ii] = [ii%N, np.floor((ii)/N), 0];
    Apos[ii] = vp.vector(ii%L, np.floor((ii)/L), 0);
    spinV[ii] = 1; # Create a container to store the values of the spins
    spinC[ii] = vp.vector(1,0,0); # Create a container to store the color code of the spins
    nIdx[ii] = nNbr(ii, L, dims);
for ii in range(int(N/2)): # Randomly choose half of the lattice to flip the spin 
    kk = choice(range(N));
    spinV[kk] = -1;
    spinC[kk] = vp.vector(1,1,1); # color all the spin down as white
ruler = np.random.rand(len(spinV));

In [ ]:
# Start the iteration and implement Metropolis algorithm
for it in range(MaxIteration):
    for ii in range(len(spinV)):
        nIdx[ii] = nNbr(ii, L, dims);
        probs[ii] = exp(sum(-beta*spinV[nIdx[ii]])*spinV[ii])/exp(sum(beta*spinV[nIdx[ii]])*spinV[ii]);
        if probs[ii] >= ruler[ii]:
            spinV[ii] = -spinV[ii];
            spinC[ii] = vp.vector(1,0,0) if spinV[ii] > 0 else vp.vector(1,1,1);
    for ii in range(N*N):
        vp.sphere(pos=Apos[ii], radius=r, color=spinV[ii])